# Connect to the api and extract the data

In [9]:
import requests
from dotenv import load_dotenv
import os

#cargamos las variables del archivo .env
load_dotenv()
key = os.getenv("KEY")

url = f'https://api.weatherapi.com/v1/forecast.json?key={key}&q=Buenos Aires&days=7&aqi=no&alerts=no'
response = requests.get(url)
data = response.json()

forecast_data_dict = {}

# Iterar sobre los datos de pronóstico
for forecast_day in data.get('forecast', {}).get('forecastday', []):
    hourly_data = forecast_day.get('hour', [])
    date = forecast_day.get('date')
    
    # Iterar sobre los datos de pronóstico cada 2 horas
    for i in range(0, len(hourly_data), 2):
        hour_data = hourly_data[i]
        time = hour_data.get('time')
        forecast_data_dict[time] = {
            'temperature_C': hour_data.get('temp_c'),
            'condition': hour_data.get('condition', {}).get('text'),
            'wind_speed_kph': hour_data.get('wind_kph'),
            'wind_direction': hour_data.get('wind_dir'),
            'pressure_mb': hour_data.get('pressure_mb'),
            'humidity_percentage': hour_data.get('humidity'),
            'cloud_cover_percentage': hour_data.get('cloud'),
            'chance_of_rain_percentage': hour_data.get('chance_of_rain'),
            'precipitation_mm': hour_data.get('precip_mm'),
            'uV_Index': hour_data.get('uv'),
        }

forecast_data_dict

{'2024-03-09 00:00': {'temperature_C': 23.5,
  'condition': 'Patchy light rain in area with thunder',
  'wind_speed_kph': 13.0,
  'wind_direction': 'ENE',
  'pressure_mb': 1007.0,
  'humidity_percentage': 79,
  'cloud_cover_percentage': 80,
  'chance_of_rain_percentage': 100,
  'precipitation_mm': 0.57,
  'uV_Index': 1.0},
 '2024-03-09 02:00': {'temperature_C': 20.7,
  'condition': 'Moderate or heavy rain shower',
  'wind_speed_kph': 18.0,
  'wind_direction': 'S',
  'pressure_mb': 1010.0,
  'humidity_percentage': 95,
  'cloud_cover_percentage': 90,
  'chance_of_rain_percentage': 100,
  'precipitation_mm': 3.32,
  'uV_Index': 1.0},
 '2024-03-09 04:00': {'temperature_C': 20.2,
  'condition': 'Mist',
  'wind_speed_kph': 10.1,
  'wind_direction': 'SE',
  'pressure_mb': 1009.0,
  'humidity_percentage': 96,
  'cloud_cover_percentage': 71,
  'chance_of_rain_percentage': 100,
  'precipitation_mm': 0.05,
  'uV_Index': 1.0},
 '2024-03-09 06:00': {'temperature_C': 20.3,
  'condition': 'Mist',
  '

In [10]:
# Crear DataFrame de Pandas
import pandas as pd
from datetime import datetime

df = pd.DataFrame.from_dict(forecast_data_dict, orient='index').reset_index().rename(columns={"index": 'hour'})

creation_date = datetime.now()
df['creation_date'] = creation_date

df

,hour,temperature_C,condition,wind_speed_kph,wind_direction,pressure_mb,humidity_percentage,cloud_cover_percentage,chance_of_rain_percentage,precipitation_mm,uV_Index,creation_date
0,2024-03-09 00:00,23.5,Patchy light rain in area with thunder,13.0,ENE,1007.0,79,80,100,0.57,1.0,2024-03-09 14:17:50.089327
1,2024-03-09 02:00,20.7,Moderate or heavy rain shower,18.0,S,1010.0,95,90,100,3.32,1.0,2024-03-09 14:17:50.089327
2,2024-03-09 04:00,20.2,Mist,10.1,SE,1009.0,96,71,100,0.05,1.0,2024-03-09 14:17:50.089327
3,2024-03-09 06:00,20.3,Mist,9.4,ENE,1008.0,96,54,0,0.00,1.0,2024-03-09 14:17:50.089327
4,2024-03-09 08:00,21.4,Partly Cloudy,5.4,NE,1009.0,94,50,0,0.00,7.0,2024-03-09 14:17:50.089327
...,...,...,...,...,...,...,...,...,...,...,...,...
79,2024-03-15 14:00,21.2,Patchy rain nearby,14.4,ESE,1007.0,88,100,89,0.00,6.0,2024-03-09 14:17:50.089327
80,2024-03-15 16:00,21.3,Patchy rain nearby,14.0,ESE,1006.0,89,98,100,0.10,6.0,2024-03-09 14:17:50.089327
81,2024-03-15 18:00,21.0,Patchy rain nearby,13.7,ESE,1005.0,90,95,100,0.13,6.0,2024-03-09 14:17:50.089327
82,2024-03-15 20:00,20.8,Patchy rain nearby,9.7,ESE,1005.0,94,98,100,0.50,1.0,2024-03-09 14:17:50.089327


# Connect and Create table in Redshift

In [11]:
df.dtypes

hour                                 object
temperature_C                       float64
condition                            object
wind_speed_kph                      float64
wind_direction                       object
pressure_mb                         float64
humidity_percentage                   int64
cloud_cover_percentage                int64
chance_of_rain_percentage             int64
precipitation_mm                    float64
uV_Index                            float64
creation_date                datetime64[us]
dtype: object

In [15]:
import psycopg2

def get_db_connection(host_instance, data_base, user, port, pwd):
    try:
        conn = psycopg2.connect(
            host=host_instance,
            dbname=data_base,
            user=user,
            password=pwd,
            port=port
        )
        print("successful connection to AWS Redshift")
        return conn
    except Exception as e:
        print("error connecting to AWS Redshift")
        print(e)

In [16]:
host_instance = os.getenv("AWS_HOST")
data_base = os.getenv("AWS_DB")
user = os.getenv("AWS_USER")
port = os.getenv("AWS_PORT")
pwd = os.getenv("AWS_PASSWORD")

# Llamamos a la funcion get_db_connection
conn = get_db_connection(host_instance, data_base, user, port, pwd)

successful connection to AWS Redshift


In [17]:
# Create Redshift table 
try:
    cur = conn.cursor()

    cur.execute("""
        CREATE TABLE IF NOT EXISTS forecastday_per_hour (
          hour VARCHAR(50),
          temperature_c FLOAT,
          condition VARCHAR(50),
          wind_speed_kph FLOAT,
          wind_direction VARCHAR(50),
          pressure_mb FLOAT,
          humidity_percentage INTEGER,
          cloud_cover_percentage INTEGER,
          chance_of_rain_percentage INTEGER,
          precipitation_mm FLOAT,
          uv_index FLOAT,
          creation_date TIMESTAMP
        )
    """)

    conn.commit()
    print("Table created successfully")
except Exception as e:
    print("Error creating table")
    print(e)

Table created successfully


In [20]:
cur=conn.cursor()
cur.execute("Truncate table forecastday_per_hour")

In [21]:
try:
    for index, row in df.iterrows():
        values = [str(value) if isinstance(value, pd.Timestamp) else value for value in row]
        insert_query = f"INSERT INTO forecastday_per_hour VALUES ({', '.join(['%s'] * len(values))});"
        cur.execute(insert_query, values)
    
    conn.commit()
    print("Data inserted successfully")
except Exception as e:
    print("Error inserting data")
    print(e)

Data inserted successfully
